<a href="https://colab.research.google.com/github/Seoyunki/Project_01/blob/main/ai13_sc43x_%EC%84%9C%EC%9C%A4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src='https://user-images.githubusercontent.com/6457691/90080969-0f758d00-dd47-11ea-8191-fa12fd2054a7.png' width = '200' align = 'right'>

## AIB / SECTION 4 / SC43x 

---



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Major Neural Network Architectures

이번 한 주간 CNN, U-Net, Autoencoder, GAN 등 다양한 주요 신경망 구조들에 대해서 배워봤습니다. 오늘은 그 모델들을 복습하는 시간을 가지도록 하겠습니다. 이 SC는 **신경망의 다양한 구조에 대한 이해와 지식**을 평가합니다. **모델을 높은 정확도를 가지도록 학습 시킬 수 있는지를 평가하려는 것이 아닙니다.**

아래의 방식들은 복잡한 연산을 요구합니다. 모든 파트의 문제들은 어떤 환경에서라도 (e.g. 로컬 주피터, Google Colab, etc.) 5-10분 내외로 결과값이 나오도록 제작이 됐기 때문에 만일 결과값을 도출하는데 그 이상의 시간이 걸린다면 여러분의 접근 방식을 재점검해보시기 바랍니다.

---

## 1. CNN

### 이미지 분류
Keras와  [ResNet50v2](https://www.tensorflow.org/api_docs/python/tf/keras/applications/resnet_v2) (pre-trained)을 활용하여 `im_frog` 폴더에 있는 이미지 중 어떤 이미지에 개구리가 있는지 찾는 이미지 분류 모델을 작동시켜 보겠습니다.

<img align="left" src="https://d3i6fh83elv35t.cloudfront.net/newshour/app/uploads/2017/03/GettyImages-654745934-1024x687.jpg" width=400>

## 2.1 Resnet$V_2$을 사용하기 위해서 전처리 함수를 사용하여 이미지를 전처리 하고 이미지들의 사이즈를 재조정하는 함수를 정의해봅시다.

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

- **파일 경로와 이름을 불러옵니다.**

In [2]:
import os

frog_dir = "/content/drive/MyDrive/im_frog"

In [4]:
"""Hint : os 패키지 내에 파일 이름을 불러올 수 있는 메서드를 사용합니다."""

filenames = os.listdir(frog_dir)

- **필요한 라이브러리 import 후 이미지를 불러와 예측하는 함수를 정의합니다.**

In [3]:
from keras.preprocessing.image import load_img, img_to_array

import numpy as np

In [40]:
import tensorflow as tf

def load_and_preprocessing(base_dir, name, model):
    """
    이미지 1장을 받아 모델로 예측한 뒤
    가장 확률이 높은 클래스 번호를 출력하는 함수입니다.
    
    Hint:
        1. 
        2. array의 값을 직접 나누어 픽셀 값을 정규화합니다.
        
    Args:
        base_dir : 이미지 파일이 있는 경로입니다.
        name : 이미지 파일의 이름입니다.
        model : 예측에 사용할 모델입니다.
    """
    image_path = base_dir
    image = load_img(image_path + '/' + name)
    input_arr = img_to_array(image) / 255
    
    input_arr = np.array([input_arr])
    ## 사진마다 size 가 달라서 resize 추가 하였음
    input_arr_resize = tf.image.resize(input_arr, (224, 224))
    predictions = model.predict(input_arr_resize)
    predict_class = np.argmax(predictions, axis=1) 
    print(predict_class[0])

    return predict_class[0]

## 2.2 ResNet50v2 모델을 사용해 이미지 분류(예측)를 진행합니다. 예측 결과는 자유롭게 출력해봅니다.
> 참고: `ResNet50v2`는 "frog"로 예측하지 않습니다. "frog"의 label은 "bullfrog, treefrog, tailed frog"입니다

In [41]:
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, decode_predictions, preprocess_input

resnetV2 = ResNet50V2(weights='imagenet', include_top = True, input_shape=(224,224,3))
##resnetV2.summary()

- **기존에 정의한 함수(`load_and_preprocessing`)를 사용하여 이미지 분류를 진행하여봅니다.**

In [42]:
# 출력되는 클래스의 번호를 모두 리스트에 저장하여 predict_class 에 할당합니다.

predict_class = [load_and_preprocessing(frog_dir, filename, resnetV2) for filename in filenames]

397
738
31
30
807
985
113
868
32
308
985
309
58
31
58


- **클래스의 이름을 다운받아 출력하여봅니다.**

In [43]:
!git clone https://github.com/anishathalye/imagenet-simple-labels.git

Cloning into 'imagenet-simple-labels'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 21 (delta 1), reused 10 (delta 1), pack-reused 10
Unpacking objects: 100% (21/21), done.


In [44]:
import json

with open ('/content/imagenet-simple-labels/imagenet-simple-labels.json') as f:
    labels = json.load(f)

for i in predict_class:
    print(labels[i])

pufferfish
pot
tree frog
American bullfrog
solar thermal collector
daisy
snail
tray
tailed frog
fly
daisy
bee
water snake
tree frog
water snake


---

## 2. U-Net

Lecture Note에서는 U-Net의 백본(backbone)모델로 `MobileNetV2`를 사용하여 segmentation을 수행하였습니다.<br/>
이번 SC에서는 ResNet50을 백본으로 하여 같은 문제를 풀어보세요.

참고로 resnet에서의 block은 아래 예시의 3개의 레이어를 참조하여 만들어주세요.<br/>
예시는 16x16 까지만 나타나 있지만 Lecture Note 와 같이 4x4까지 만들어 주어야 모델을 완성할 수 있습니다.

```
    'conv1_relu', # 64x64
    'conv2_block3_out', # 32x32 
    'conv3_block4_out', # 16x16
```

In [55]:
## back bone 모델인 ResNet50을 불러온다.

base_model = tf.keras.applications.ResNet50(input_shape=[128, 128, 3], include_top=False)

base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 134, 134, 3)  0           ['input_7[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 64, 64, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 64, 64, 64)   256         ['conv1_conv[0][0]']      

In [47]:
## 1) Downsamling 부분 만들기

layer_names = [
    'conv1_relu',         # 64x64x3
    'conv2_block3_out',   # 32x32x64
    'conv3_block4_out',   # 16x16x256
    'conv4_block5_out',   # 8x8x512
    'conv5_block3_out',   # 4x4x2048
]


layers = [base_model.get_layer(name).output for name in layer_names]

down_stack = tf.keras.Model(inputs=base_model.input, outputs=layers)

down_stack.trainable = False

In [50]:
# pix2pix 를 사용하기 위해 package 다운로드한다.
!pip install -q git+https://github.com/tensorflow/examples.git

    Running setup.py install for tensorflow-examples ... done
  DEPRECATION: tensorflow-examples was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. You can find discussion regarding this at https://github.com/pypa/pip/issues/8368.


In [51]:
## 2) Upsamling 부분을 만든다. 

from tensorflow_examples.models.pix2pix import pix2pix

up_stack = [
    pix2pix.upsample(512, 3),  # 4x4 -> 8x8
    pix2pix.upsample(256, 3),  # 8x8 -> 16x16
    pix2pix.upsample(128, 3),  # 16x16 -> 32x32
    pix2pix.upsample(64, 3),   # 32x32 -> 64x64
]

In [52]:
## 3) U-Net 모델을 정의한다.

def unet_model(output_channels):
    inputs = tf.keras.layers.Input(shape=[128, 128, 3])
    x = inputs

    # Downsampling 함수 호출
    skips = down_stack(x)
    x = skips[-1]
    skips = reversed(skips[:-1])

    # Upsampling 함수 호출
    for up, skip in zip(up_stack, skips):
        x = up(x)
        # Downsampling 결과를 Concatenate 해줍니다.
        concat = tf.keras.layers.Concatenate()
        x = concat([x, skip])

    # 입력과 shape 맞추기위해 마지막 layer 추가
    last = tf.keras.layers.Conv2DTranspose(
        output_channels, 3, strides=2,
        padding='same')  #64x64 -> 128x128

    x = last(x)

    return tf.keras.Model(inputs=inputs, outputs=x)

In [54]:
## 4) 만든 모델을 compile 하고, summary 로 확인
OUTPUT_CHANNELS = 3

model = unet_model(OUTPUT_CHANNELS)

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 model (Functional)             [(None, 64, 64, 64)  23587712    ['input_6[0][0]']                
                                , (None, 32, 32, 25                                               
                                6),                                                               
                                 (None, 16, 16, 512                                               
                                ),                                                          

## 3. Free Response
> 동료들에게 설명한다고 생각하고 간단하게 몇 문장으로 작성하세요.

### 3.1 딥러닝이 왜 중요하다고 생각하시나요?

그동안 이론적으로만 가능하고 실제 구현할 경우 성능이 떨어졌었던 딥러닝 분아가 GPU 와 같은 하드웨어의 비약적인 발전과 알고리즘의 끊임 없는 개선으로 뛰어난 성능을 보여주고 있습니다.   

게다가 최근의 결과들은 하드웨어의 사양이 좋아지면 좋아질 수록 병렬처리에 의해 해당 머신러닝 모델의 성능도 비례해 증가하는 경향이 있어, 앞으로 그 발전 가능성이 얼마나 더 남아있을지 가늠이 안되는 상황입니다.

### 3.2 딥러닝의 다양한 분야 중에서 좀 더 심도있게 배우고 싶은 분야는 무엇인가요? 왜 그렇게 생각하시나요?

이미지, 음성 분야에 대해 더 심도 있게 학습해보고 싶습니다. 

사람이 가지고 있는 오감인 시각, 청각, 촉각, 미각, 후각중 삶의 질에 매우 큰 영향을 미치는 것은 시각, 청각이 압도적이라고 생각됩니다. 보고 들을 수 없다면 인간과의 관계가 단절될 가능성이 높기 때문입니다. 물론 시각 및 청각 장애인들은 부단한 노력으로 주위와의 관계를 이어가고 있지만, 딥러닝을 통해 이들을 도와줄 수 있다면 인권이란 측면에서도 큰 의미가 있을 것 같습니다.

또한 정상인의 경우에도 시각, 청각을 머신러닝을 통해 보조할 수 있다면 사람이기에 가지고 있는 실수의 가능성을 줄이고 이로인해 실수로 인한 손해 비용(기회)을 최소화 할 수 있을 것입니다.

### 3.3 인공지능이 우리 사회에서 대체할 수 있는 직업이 무엇이라고 생각하시나요?


감정을 소모하는 직업들, 반복되는 동작을 수행하는 작업들 등이 사라질 것 같습니다.

이미 상담원과 같은 감정 근로자들은 AI 상담원으로 빠르게 대체되고 있습니다. 또한 제조업에서의 무인화는 이미 상당히 진행된 상황인데, 최근 인공지능의 비약적인 발전에 따라 독일의 경우 industry 4.0 을 강력하게 추진하면서 철강 라인의 경우 operation room 에 있는 2~3명의 인원이 과거 수십명이 일하던 라인을 가동하는 모습을 보여주고 있습니다. 또한 테슬라 및 나이키 등에서도 smart factory 를 구축해 근로자가 거의 없이 무인에 가까운 공장 가동을 구현해내고 있습니다.

boston dynamics 에서의 로봇 개발은 이미 상당한 수준이며 , 가격이라는 걸림돌이(4족 보행로봇 대당 가격이 1억원이 넘음) 해결된다면 산업현장에서의 사람->로봇 대체는 더욱 가속화 될 것입니다.

### 3.4 반대로 인공지능 때문에, 딥러닝 때문에 더 생겨날 직업은 무엇이 있을까요?


해당 머신러닝을 만들고, 유지 보수하는 직업(빅데이터라고 불리우는 대용량의 데이터를 보관하기위한 서버의 구성 및 운영을 포함)과 머신러닝이 적용된 분야에 대해 교육을 통해 인재를 길러내는 교육 관련 계통 등이 더 활발하게 창출 될 것 같습니다.

구글, MS 등에서 인공지능에 대해 지속적으로 개발하고 그 수준을 끌어올리고 있지만, 결국 사용되는 개소에 맞추어야 하는 문제점이 있습니다. 사용 개소에 대해 가장 잘 아는 사람은 해당 개소의 사업자 및 사업장내 근로자들일 것입니다. 기술의 이식, 이식이후 재학습, 데이터 관리 등은 모두 외주화하기도 어렵고 결국 비용의 문제와 연결되기때문에 경영진 입장에서는 관련 인력을 육성하게 될 것으로 보입니다. 
우리나라의 경우 대기업군 -> 중견기업군 순으로 기술이 전파되어나가는 모습을 보여주는데, 위에서도 기술한 비용이라는 턱이 낮아진다면 중견 -> 중소 기업으로 넘어가는 것도 시간 문제라고 생각됩니다.

### 3.5 여러분이 생각하실 때 Strong AI라고 불리우는 [일반 인공지능 (Artificial General Intelligence)](https://ko.wikipedia.org/wiki/%EC%9D%B8%EA%B3%B5_%EC%9D%BC%EB%B0%98_%EC%A7%80%EB%8A%A5)을 개발해내는 것이 가능할 것 같나요? 왜 그렇게 생각하시나요?

하드웨어가 어느정도까지 발전할 것인지에 달렸다고 볼 수 있을 것 같습니다.

연산 처리속도, 처리량이 이론적으로 필요하다고 하는 수준을 만족한다면 결국 Strong AI 를 필요로 하는 곳에서 대규모 투자가 따라올 것이라고 생각됩니다.

특히 하드웨어 쪽은 반도체 분야에서의 무어의 법칙이 여전히 유효하고, 이런 추세라면 최근 급속도로 발전하고 있는 클라우드 분야가 일반 기업은 물론 개인 가정까지도 진출할 수 있을 것으로 보는 이야기도 있습니다. 이러한 하드웨어의 발전을 바탕으로 (규모의 발전을 포함함) 연쇄적으로 AI 의 발전은 그 한계를 향해 계속 진행해갈 것으로 생각됩니다.

### 3.6 Coutinous learning에 대해서 조사해보고 인지한 부분에 대해서 작성해보세요.

Continuous learning 이란 새로운 기술, 지식을 계속해서 배워나가는 과정입니다.정규교육에서 비정규교육에 걸친 여러가지 방법이 있을 것이고, 어떤 기관에서 이루어질 수도 있고, 개인적으로도 이루어질 수 있을 것입니다.

하지만 그 중심에는 스스로 동기를 부여하고, 목표를 세워 이를 달성하는 과정이 계속해서 이어진다는 점이 (life-time 의 주기로) 포함되어야 하겠습니다.

# Advanced Goals: 3점을 받기 위해서는 아래의 조건 중에서 3개 이상을 달성하셔야 합니다

### 1
    - 개구리 이외의 다른 객체를 탐지하는 모델을 만들어보세요 (예: 물고기)
    - 이미지를 예측한 label과 같이 출력해보세요
    - 예측 모델을 함수로 만들어 보세요 (물론 주석도 잘 되어있어야 합니다)
### 2
    - U-Net 을 직접 구현하여 동일한 문제를 수행해보세요. 
### 3
    - 작성한 답안을 몇 문장보다 조금 더 상세하게 작성해보세요
    - 왜 그렇게 생각하게 되었는 지 관련 근거를 서술하세요.
### 추가과제
    - GAN을 이용한 프로젝트를 새롭게 구현해보세요.

In [ ]:
## 고양이를 탐지하는 모델을 만들어 봅니다.
## 이미지는 총 11개
## 진짜 고양이는 9마리
## 진짜 고양이중 2마리는 사자 및 표범을 닮아서 사람도 헛갈림
## 진짜 고양이중 2마리는 가방 및 상자에 들어가 있음
## 진짜 고양이중 3마리는 수풀에 숨어있음 
## 강아지 2마리 ; 고양이를 닮은 느낌의 강아지


In [57]:
## 구글 드라이브에 이미지 11개를 업로드했고, 해당 경로를 받는다.
cat_dir = "/content/drive/MyDrive/im_cat"
filenames = os.listdir(cat_dir)

In [58]:
## 이미지를 불러와 전처리하는 함수를 정의한다.
def load_and_preprocessing(base_dir, name, model):
    image_path = base_dir
    image = load_img(image_path + '/' + name)
    input_arr = img_to_array(image) / 255
    
    input_arr = np.array([input_arr])
    ## 사진마다 size 가 달라서 resize 추가 하였음
    input_arr_resize = tf.image.resize(input_arr, (224, 224))
    predictions = model.predict(input_arr_resize)
    predict_class = np.argmax(predictions, axis=1) 
    print(predict_class[0])

    return predict_class[0]

In [59]:
## ResNet50V2를 불러온다.
resnetV2 = ResNet50V2(weights='imagenet', include_top = True, input_shape=(224,224,3))
##resnetV2.summary()

In [65]:
## 각 파일을 불러와 전처리한 이미지들을 사용자 함수에 넣어 예측 label을 저장한다.
predict_set = {}
#predict_class = [load_and_preprocessing(cat_dir, filename, resnetV2) for filename in filenames]

for filename in filenames:
  predicted_class = load_and_preprocessing(cat_dir, filename, resnetV2)
  predict_set[filename] = predicted_class

predict_set

333
281
281
23
285
882
157
272
285
227
335


{'15340d90-0449-11ed-bf8f-adafb7e111f1.cf.jpg': 882,
 'Cat-Names-Inspired-By-Royalty-g.jpg': 281,
 'Cat_on_the_tree_3.jpg': 335,
 'Does-this-dog-really-look-like-a-cat.jpg': 227,
 'Ew_0482WQAQ9U7e.jpg': 333,
 'amazing-black-cat-looking-green-leaves-beautiful-dark-cat-green-eyes-standing-bush-leaves-hiding-looking-113181220.jpg': 23,
 'cats-that-look-like-tigers-leopards-and-cheetahs.jpg': 285,
 'd41586-022-01792-y_23218334.jpg': 281,
 'dog-breeds-that-are-like-cats-papillon-1581448671.jpg': 157,
 'faworek__cat_in_bush_by_nefkacolorshow_db6eg3m-fullview.jpg': 285,
 'grey-cat-in-the-tree.jpg': 272}

In [66]:
import json

## 정의된 label 을 불러오고, 예측값의 label 을 출력한다.
with open ('/content/imagenet-simple-labels/imagenet-simple-labels.json') as f:
    labels = json.load(f)

##for i in predict_class:
##    print(labels[i])

## 파일 이름과 예측 label 을 묵어서 출력해봄
for key, value in predict_set.items():
  print('key : {},     predict label : {}'.format(key, labels[value]))


key : Ew_0482WQAQ9U7e.jpg,     predict label : hamster
key : d41586-022-01792-y_23218334.jpg,     predict label : tabby cat
key : Cat-Names-Inspired-By-Royalty-g.jpg,     predict label : tabby cat
key : amazing-black-cat-looking-green-leaves-beautiful-dark-cat-green-eyes-standing-bush-leaves-hiding-looking-113181220.jpg,     predict label : vulture
key : cats-that-look-like-tigers-leopards-and-cheetahs.jpg,     predict label : Egyptian Mau
key : 15340d90-0449-11ed-bf8f-adafb7e111f1.cf.jpg,     predict label : vacuum cleaner
key : dog-breeds-that-are-like-cats-papillon-1581448671.jpg,     predict label : Papillon
key : grey-cat-in-the-tree.jpg,     predict label : coyote
key : faworek__cat_in_bush_by_nefkacolorshow_db6eg3m-fullview.jpg,     predict label : Egyptian Mau
key : Does-this-dog-really-look-like-a-cat.jpg,     predict label : Australian Kelpie
key : Cat_on_the_tree_3.jpg,     predict label : fox squirrel


이미지 판정 결과

파일 이름 : Ew_0482WQAQ9U7e.jpg(상자에 들어가있는 고양이)  => 판정 : hamster 

파일 이름 : d41586-022-01792-y_23218334.jpg(목에 이름표를 차고 있는 고양이 ; tabby cat) => 판정 : tabby cat

파일 이름 : Cat-Names-Inspired-By-Royalty-g.jpg (사자 닮은 고양이) => 판정 : tabby cat (고양이는 맞는데 종을 다르게 판정)

파일 이름 : amazing-black-cat-looking-green-leaves-beautiful-dark-cat-green-eyes-standing-bush-leaves-hiding-looking-113181220.jpg (수풀에 숨어있는 검은색 고양이)
   => 판정 : vulture (대머리 수리)

파일 이름 : cats-that-look-like-tigers-leopards-and-cheetahs.jpg (사자를 닮은 고양이) => 판정 : Egyptian Mau (고양이 종도 맞게 예측)

파일 이름 : 15340d90-0449-11ed-bf8f-adafb7e111f1.cf.jpg (가방에 담겨서 머리만 내놓고 있는 고양이) => 판정 : vacuum cleaner (멀리서 보면 진공청소기 같기도 함)

파일 이름 : dog-breeds-that-are-like-cats-papillon-1581448671.jpg (고양이와 비슷하게 생긴 강아지) => 판정 : Papillon (강아지 종도 정확히 판정)

파일 이름 : grey-cat-in-the-tree.jpg (나무 위에 올라가 있는 고양이) => 판정 : coyote

파일 이름 : faworek__cat_in_bush_by_nefkacolorshow_db6eg3m-fullview.jpg (수풀에 숨어 있는 고양이) -> 판정 : Egyptian Mau

파일 이름 : Does-this-dog-really-look-like-a-cat.jpg (고양이 닮은 강아지) => 판정 : Australian Kelpie (강아지로 판정했으나 종은 다름)

파일 이름 : Cat_on_the_tree_3.jpg (나무위에 올라가 있는 고양이) => fox squirrel (사진을 조금 작게해서 보면 여우 느낌이 나는데, 여우 다람쥐로 판정함; 여우다람쥐는 딱히 여우와 닮지는 않음)
